<center><b>In the name of God</b></center>

<b>Course</b>: Machine Learning
<br>
<b>Description:</b> HomeWork 2 | Question 1
<br>
<b>Developer</b>: Alireza Mazochi (400131075)

# Dataset Information

<b>Name</b>: Heart Disease Dataset<br>
<b>Refrence</b>: https://www.kaggle.com/johnsmith88/heart-disease-dataset

## Context

This data set dates from 1988 and consists of four databases: Cleveland, Hungary, Switzerland, and Long Beach V. It contains 76 attributes, including the predicted attribute, but all published experiments refer to using a subset of 14 of them. The "target" field refers to the presence of heart disease in the patient. It is integer valued 0 = no disease and 1 = disease.

## Content

<nl>
    <li>age: age in years</li>
    <li>sex: (1 = male; 0 = female)</li>
    <li>cp: chest pain type (4 values)</li>
    <li>restcg: resting blood pressure (in mm Hg on admission to the hospital)</li>
    <li>chol: serum cholestoral in mg/dl</li>
    <li>fbs: fasting blood sugar > 120 mg/dl</li>
    <li>restecg: resting electrocardiographic results (values 0,1,2)</li>
    <li>thalach: maximum heart rate achieved</li>
    <li>exang: exercise induced angina(1 = yes; 0 = no)</li>
    <li>oldpeak: ST depression induced by exercise relative to rest</li>
    <li>slope: the slope of the peak exercise ST segment</li>
    <li>ca: number of major vessels (0-3) colored by flourosopy</li>
    <li>thal: 0 = normal; 1 = fixed defect; 2 = reversable defect</li>
</nl>


# Libraries

In [1]:
# Import Allowed Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load and Overview

In [2]:
data = pd.read_csv("heart.csv")

In [3]:
data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


# Train and Test Split

In [4]:
data_train = data.head(int(0.8*len(data)))
data_test = data.tail(len(data) - int(0.8*len(data)))

In [5]:
data_train_x = data_train.drop(columns=["target"])
data_train_y = data_train[["target"]]

# Prior Probability

In [6]:
classes = [0, 1]

In [7]:
p_prior = np.zeros(2)
for class_ in classes:
    p_prior[class_] = len(data_train_y[data_train_y["target"]==class_]) / len(data_train_y)

# Discreate and Continuous Features

In [8]:
discrete_features = ["sex", "cp", "fbs", "restecg", "exang", "slope", "ca", "thal"]
continuous_features = ["age", "trestbps", "chol", "thalach", "oldpeak"]

In [9]:
# Some checking
for feature in discrete_features:
    if len(set(data[feature].values)) > 5:
        print(set(data[feature].values))
        raise Exception(f"It seems to feature {feature} isn't a discrete feature")
        
for feature in continuous_features:
    if len(set(data[feature].values)) < 20:
        print(set(data[feature].values))
        raise Exception(f"It seems to feature {feature} isn't a continous feature")

# Likelihood Probability

In [10]:
p_likelihood_discrete = []
discrete_features_value_number = dict()

for class_ in classes:
    p_likelihood_discrete.append(dict())
    for feature in discrete_features:
        discrete_features_value_number[feature] = len(set(data[feature].values))
        p_likelihood_discrete[class_][feature] = np.zeros(discrete_features_value_number[feature])
    
for class_ in classes:
    data_class = data_train[data_train["target"]==class_]
    for feature in discrete_features:
        for value in range(discrete_features_value_number[feature]):
            p_likelihood_discrete[class_][feature][value] = len(data_class[data_class[feature]==value]) / len(data_class)

In [16]:
p_likelihood_continuous_independent = []

for class_ in classes:
    data_class = data_train[data_train["target"]==class_]

    p_likelihood_continuous_independent.append(dict())
    for feature in continuous_features:
        p_likelihood_continuous_independent[class_][feature] = {"mean": data_class[feature].mean(), "var": data_class[feature].var()}